In [1]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline
import numpy as np
import string
import os
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel


from xgboost import XGBRegressor

In [2]:
os.path.abspath('..')
sys.path.append(os.path.abspath('..'))


In [3]:
import importlib


import board_helpers
from board_helpers.move_info import MoveInfo
from board_helpers.tiles_counter import TilesCoutner
from board_helpers.board_consts import TileType
importlib.reload(board_helpers)
importlib.reload(board_helpers.move_info)
importlib.reload(board_helpers.tiles_counter)
importlib.reload(board_helpers.board_consts)



import transformers
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_rows_transformer import SelectRowsTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer
from transformers.one_hot_encoder_transformer import OneHotEncoderTransformer
from transformers.add_to_dict_transformer import AddToDictTransformer
from transformers.get_from_dict_transformer import GetFromDictTransformer
from transformers.select_pos_rows_transformer import SelectPosRowsTransformer
from transformers.select_features_wrapper import SelectFeaturesWrapper
importlib.reload(transformers)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_rows_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)
importlib.reload(transformers.one_hot_encoder_transformer)
importlib.reload(transformers.add_to_dict_transformer)
importlib.reload(transformers.get_from_dict_transformer)
importlib.reload(transformers.select_pos_rows_transformer)
importlib.reload(transformers.select_features_wrapper)

import functions
from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions)
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)

import processors
from processors.basic_pre_processor import BasicPreProcessor
importlib.reload(processors)
importlib.reload(processors.basic_pre_processor)

import builders
from builders.preprocessor_builder import PreprocessorBuilder
from builders.reg_pipe_builder import RegPipeBuilder
importlib.reload(builders)
importlib.reload(builders.preprocessor_builder)
importlib.reload(builders.reg_pipe_builder)

import searcher
import searcher_results_orgenizer
from searcher import Searcher
from searcher_results_orgenizer import SearcherResultsOrgenizer
importlib.reload(searcher)
importlib.reload(searcher_results_orgenizer)







<module 'searcher_results_orgenizer' from 'c:\\Users\\guyv2\\OneDrive\\שולחן העבודה\\hafifa\\scrabble\\searcher_results_orgenizer.py'>

In [4]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
a_turns = pd.read_csv('../data/a_new_extracted_features_turns.csv', index_col='game_id')
p_turns = pd.read_csv('../data/p_new_extracted_features_turns.csv', index_col='game_id')
b_turns = pd.read_csv('../data/b_new_extracted_features_turns.csv', index_col='game_id')

In [5]:
a_turns.columns = 'a_' + a_turns.columns

In [6]:
a_turns

,a_points__sum_values,a_points__median,a_points__mean,a_points__length,a_points__standard_deviation,a_points__variance,a_points__root_mean_square,a_points__maximum,a_points__absolute_maximum,a_points__minimum,...,a_special_tile__sum_values,a_special_tile__median,a_special_tile__mean,a_special_tile__length,a_special_tile__standard_deviation,a_special_tile__variance,a_special_tile__root_mean_square,a_special_tile__maximum,a_special_tile__absolute_maximum,a_special_tile__minimum
game_id,,,,,,,,,,,,,,,,,,,,,
1,764.0,24.0,28.296296,27.0,20.956418,439.171468,35.211530,98.0,98.0,2.0,...,33.0,1.0,1.222222,27.0,0.684935,0.469136,1.401058,3.0,3.0,0.0
2,889.0,24.5,34.192308,26.0,27.447880,753.386095,43.846323,94.0,94.0,0.0,...,35.0,1.0,1.346154,26.0,0.730769,0.534024,1.531716,3.0,3.0,0.0
3,758.0,25.0,28.074074,27.0,19.584434,383.550069,34.230158,103.0,103.0,0.0,...,34.0,1.0,1.259259,27.0,0.583260,0.340192,1.387777,2.0,2.0,0.0
4,597.0,16.0,20.586207,29.0,19.526317,381.277051,28.373737,74.0,74.0,0.0,...,35.0,1.0,1.206897,29.0,0.609087,0.370987,1.351882,2.0,2.0,0.0
5,752.0,21.0,24.258065,31.0,16.688740,278.514048,29.444316,81.0,81.0,0.0,...,38.0,1.0,1.225806,31.0,0.657938,0.432882,1.391217,3.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72769,1010.0,28.0,32.580645,31.0,43.210493,1867.146722,54.116958,194.0,194.0,-78.0,...,35.0,1.0,1.129032,31.0,0.706739,0.499480,1.331989,3.0,3.0,0.0
72770,687.0,11.5,21.468750,32.0,24.590375,604.686523,32.643433,106.0,106.0,-10.0,...,37.0,1.0,1.156250,32.0,0.666585,0.444336,1.334635,2.0,2.0,0.0
72771,783.0,20.0,25.258065,31.0,21.527129,463.417274,33.187153,93.0,93.0,0.0,...,35.0,1.0,1.129032,31.0,0.706739,0.499480,1.331989,3.0,3.0,0.0


In [7]:
G_NAME = 'games'
T_NAME = 'turns'
DATA_NAME = 'train'

names = ['BetterBot', 'STEEBot', 'HastyBot']

prePipe = Pipeline([
                    ('get_relavent_turns', SelectRowsTransformer(train.index.unique(), target=T_NAME)),
                     ('get_relavent_games',  SelectRowsTransformer(train.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectPosRowsTransformer('is_player', target=DATA_NAME)),
                    ])

In [8]:
preprocessor = PreprocessorBuilder(games, G_NAME, a_turns, T_NAME, prePipe).build()
n_games, a_n_turns, n_ratings = preprocessor.process(train, DATA_NAME)
p_n_turns = p_turns.loc[train.index.unique(),:]
b_n_turns = b_turns.loc[train.index.unique(),:]
n_games

,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,bot_rating,bot_name
game_id,,,,,,,,,,,,,
1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,1637,BetterBot
3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,2071,BetterBot
4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,1936,BetterBot
5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,1844,STEEBot
6,HivinD,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,2143,STEEBot
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,BetterBot,regular,TIME,0,2022-09-05 04:35:56,CSW21,3600,0,CASUAL,1,3669.988413,1952,BetterBot
72770,HastyBot,regular,TIME,0,2022-09-04 00:13:24,CSW21,3600,0,RATED,1,3662.659287,2237,HastyBot
72771,HastyBot,regular,STANDARD,0,2022-09-19 03:28:44,ECWL,900,0,CASUAL,10,236.133634,1614,HastyBot


In [9]:
n_data = n_games.merge(a_n_turns, left_index=True, right_index=True).merge(p_n_turns, left_index=True, right_index=True).merge(b_n_turns, left_index=True, right_index=True)

In [10]:
n_data

,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,...,b_special_tile__sum_values,b_special_tile__median,b_special_tile__mean,b_special_tile__length,b_special_tile__standard_deviation,b_special_tile__variance,b_special_tile__root_mean_square,b_special_tile__maximum,b_special_tile__absolute_maximum,b_special_tile__minimum
game_id,,,,,,,,,,,,,,,,,,,,,
1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,...,18.0,1.0,1.384615,13.0,0.737820,0.544379,1.568929,3.0,3.0,0.0
3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,...,17.0,1.0,1.307692,13.0,0.605693,0.366864,1.441153,2.0,2.0,0.0
4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,...,23.0,2.0,1.533333,15.0,0.498888,0.248889,1.612452,2.0,2.0,1.0
5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,...,22.0,1.0,1.466667,15.0,0.618241,0.382222,1.591645,3.0,3.0,1.0
6,HivinD,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,...,15.0,1.0,1.153846,13.0,0.863459,0.745562,1.441153,3.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,BetterBot,regular,TIME,0,2022-09-05 04:35:56,CSW21,3600,0,CASUAL,1,...,23.0,1.0,1.437500,16.0,0.609175,0.371094,1.561249,3.0,3.0,1.0
72770,HastyBot,regular,TIME,0,2022-09-04 00:13:24,CSW21,3600,0,RATED,1,...,22.0,1.5,1.375000,16.0,0.695971,0.484375,1.541104,2.0,2.0,0.0
72771,HastyBot,regular,STANDARD,0,2022-09-19 03:28:44,ECWL,900,0,CASUAL,10,...,17.0,1.0,1.133333,15.0,0.718022,0.515556,1.341641,3.0,3.0,0.0


In [11]:
featureTransformers = [('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                       ('hot', OneHotEncoderTransformer({'time_control_name': games['time_control_name'].unique(),
                                                          'game_end_reason':  games['game_end_reason'].unique(),
                                                          'lexicon':  games['lexicon'].unique(),
                                                          'rating_mode':  games['rating_mode'].unique(),
                                                          'bot_name': names}
                                                        )),
                       ('drops', NameDropperTransformer(['first', 'created_at', 'time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name'])),                                              
                      ]

In [12]:
reg_pipe = Pipeline(featureTransformers)
t_data = reg_pipe.transform(n_data)

In [13]:
t_data

,winner,initial_time_seconds,increment_seconds,max_overtime_minutes,game_duration_seconds,bot_rating,a_points__sum_values,a_points__median,a_points__mean,a_points__length,...,lexicon_NSWL20,lexicon_NWL20,lexicon_nan,rating_mode_CASUAL,rating_mode_RATED,rating_mode_nan,bot_name_BetterBot,bot_name_HastyBot,bot_name_STEEBot,bot_name_nan
1,1,1200,0,1,674.844274,1637,764.0,24.0,28.296296,27.0,...,0,1,0,1,0,0,1,0,0,0
3,1,900,0,5,492.268262,2071,758.0,25.0,28.074074,27.0,...,0,0,0,0,1,0,1,0,0,0
4,0,3600,0,1,350.861141,1936,597.0,16.0,20.586207,29.0,...,0,0,0,1,0,0,1,0,0,0
5,0,1200,0,1,642.688722,1844,752.0,21.0,24.258065,31.0,...,0,1,0,1,0,0,0,0,1,0
6,0,900,0,1,426.950541,2143,805.0,28.0,32.200000,25.0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,0,3600,0,1,3669.988413,1952,754.0,20.0,23.562500,32.0,...,0,0,0,1,0,0,1,0,0,0
72770,0,3600,0,1,3662.659287,2237,687.0,11.5,21.468750,32.0,...,0,0,0,0,1,0,0,1,0,0
72771,0,900,0,10,236.133634,1614,783.0,20.0,25.258065,31.0,...,0,0,0,1,0,0,0,1,0,0
72772,0,900,0,1,681.370077,1674,830.0,21.5,27.666667,30.0,...,0,1,0,0,1,0,1,0,0,0


In [14]:
grid = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0,),
    'param_grid':
        {
        },
    'scores': ['neg_root_mean_squared_error']
    },
}

In [16]:
my_searcher = Searcher(reg_pipe, grid, None, redo_scores=True)
my_searcher.search(n_data, n_ratings)

In [17]:
res = SearcherResultsOrgenizer(redo_scores=True).orgenize(my_searcher.get_results())

In [13]:
# res

In [18]:
thresholds = my_searcher.get_results()[('xgboost', 'neg_root_mean_squared_error')].best_estimator_[-1].feature_importances_
thresholds = np.sort(thresholds)
model =  my_searcher.get_results()[('xgboost', 'neg_root_mean_squared_error')].best_estimator_[-1]


In [19]:
thresholds.shape

(2169,)

In [20]:
s_featureTransformers = [('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                       ('hot', OneHotEncoderTransformer({'time_control_name': games['time_control_name'].unique(),
                                                          'game_end_reason':  games['game_end_reason'].unique(),
                                                          'lexicon':  games['lexicon'].unique(),
                                                          'rating_mode':  games['rating_mode'].unique(),
                                                          'bot_name': names}
                                                        )),
                       ('drops', NameDropperTransformer(['first', 'created_at', 'time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name'])),
                       ('selection', SelectFromModel(XGBRegressor(random_state=0))),
                       ('xgboost', XGBRegressor(random_state=0)),                                          
                      ]

s_reg_pipe = Pipeline(s_featureTransformers)


In [16]:
# model.feature_importances_

In [21]:
parameters = {'selection__threshold': thresholds[1620::50]}
# parameters = {'selection__threshold': [0.01]}


In [22]:
clf = GridSearchCV(s_reg_pipe, parameters, n_jobs=2, scoring='neg_root_mean_squared_error')


In [23]:
clf.fit(n_data, n_ratings)

GridSearchCV(estimator=Pipeline(steps=[('extract_first',
                                        ColumnsSetterTransformer(columns={'is_bot_first': <functions.is_bot_extractor.IsBotExtarctor object at 0x0000017E6D283CD0>})),
                                       ('hot',
                                        OneHotEncoderTransformer(columns={'bot_name': ['BetterBot',
                                                                                       'STEEBot',
                                                                                       'HastyBot'],
                                                                          'game_end_reason': array(['STANDARD', 'RESIGNED', 'TIME', 'CONSECUTIVE_ZEROES'], dtype=obj...
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     n_estimators=100,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     predictor=None,
                                                     random_state=0, ...))]),
             n_jobs=2,
             param_grid={'selection__threshold': array([0.0002566 , 0.00048531, 0.00058546, 0.00067749, 0.00074209,
       0.0008306 , 0.00090916, 0.00100463, 0.00112597, 0.00134   ,
       0.00205833], dtype=float32)},
             scoring='neg_root_mean_squared_error')

In [24]:
pd.DataFrame(clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_selection__threshold,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,186.332610,3.364467,1.115000,0.085095,0.000257,{'selection__threshold': 0.0002565952},-110.346743,-111.709346,-112.481544,-111.812712,-111.502710,-111.570611,0.694280,11
1,175.500303,3.431209,1.037000,0.033805,0.000485,{'selection__threshold': 0.0004853116},-111.188730,-111.837706,-111.915582,-111.715452,-110.448476,-111.421189,0.548582,9
2,171.938799,1.971218,1.026999,0.045961,0.000585,{'selection__threshold': 0.0005854625},-111.978966,-111.269317,-111.844540,-111.688166,-110.425872,-111.441372,0.560946,10
3,167.099599,1.323945,0.982200,0.029398,0.000677,{'selection__threshold': 0.00067749276},-110.439480,-111.316384,-113.566691,-110.552220,-110.809111,-111.336777,1.155166,7
4,163.809200,1.257973,0.951001,0.050200,0.000742,{'selection__threshold': 0.00074209186},-110.639931,-110.823096,-112.999053,-110.917720,-109.365783,-110.949117,1.168045,6
5,160.521199,1.302682,0.982401,0.104829,0.000831,{'selection__threshold': 0.00083060155},-110.897898,-111.786289,-112.575049,-111.548972,-110.215662,-111.404774,0.800849,8
6,159.064136,5.526208,0.952799,0.030102,0.000909,{'selection__threshold': 0.00090915937},-110.472444,-110.944015,-112.001266,-110.912189,-109.826472,-110.831277,0.710848,5
7,162.247198,3.264204,0.970403,0.064729,0.001005,{'selection__threshold': 0.0010046345},-109.750389,-111.119673,-111.733631,-109.559506,-109.707824,-110.374205,0.883256,4
8,154.146142,1.495951,1.021602,0.074991,0.001126,{'selection__threshold': 0.0011259667},-108.915146,-110.824079,-111.031420,-110.411515,-107.952195,-109.826871,1.194389,3
9,148.874246,2.134327,0.967083,0.030123,0.00134,{'selection__threshold': 0.001339998},-107.904549,-109.156528,-111.387605,-109.518235,-108.162256,-109.225835,1.236167,2


In [25]:
thresholds[1620 + 50*10]

0.0020583312

In [26]:
1620 + 50*10

2120

In [28]:
thresholds[2120 - 100::20]

array([0.00112597, 0.00119991, 0.00130941, 0.00141778, 0.00167678,
       0.00205833, 0.00370816, 0.01621042], dtype=float32)

In [21]:
parameters2 = {'selection__threshold': thresholds[2120 - 100::20]}


In [22]:
clf2 = GridSearchCV(s_reg_pipe, parameters2, n_jobs=2, scoring='neg_root_mean_squared_error')


In [23]:
clf2.fit(n_data, n_ratings)

GridSearchCV(estimator=Pipeline(steps=[('extract_first',
                                        ColumnsSetterTransformer(columns={'is_bot_first': <functions.is_bot_extractor.IsBotExtarctor object at 0x000001D7A68D2C70>})),
                                       ('hot',
                                        OneHotEncoderTransformer(columns={'bot_name': ['BetterBot',
                                                                                       'STEEBot',
                                                                                       'HastyBot'],
                                                                          'game_end_reason': array(['STANDARD', 'RESIGNED', 'TIME', 'CONSECUTIVE_ZEROES'], dtype=obj...
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     n_estimators=100,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     predictor=None,
                                                     random_state=0, ...))]),
             n_jobs=2,
             param_grid={'selection__threshold': array([0.00112597, 0.00119991, 0.00130941, 0.00141778, 0.00167678,
       0.00205833, 0.00370816, 0.01621042], dtype=float32)},
             scoring='neg_root_mean_squared_error')

In [24]:
pd.DataFrame(clf2.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_selection__threshold,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,169.516308,9.362701,1.177223,0.074061,0.001126,{'selection__threshold': 0.0011259667},-108.915146,-110.824079,-111.031420,-110.411515,-107.952195,-109.826871,1.194389,6
1,169.305267,2.466794,1.139030,0.257626,0.0012,{'selection__threshold': 0.0011999056},-108.922453,-109.560068,-111.044607,-110.394824,-108.899475,-109.764285,0.840843,5
2,152.538513,3.574462,1.059601,0.141277,0.001309,{'selection__threshold': 0.001309412},-108.455547,-109.624815,-110.528090,-109.611750,-107.770347,-109.198110,0.970765,4
3,147.520899,0.736071,1.031020,0.087672,0.001418,{'selection__threshold': 0.0014177813},-107.699033,-109.189134,-110.593914,-108.939884,-108.169383,-108.918269,0.992873,3
4,144.560800,0.584429,0.921400,0.058112,0.001677,{'selection__threshold': 0.0016767832},-107.648911,-108.895245,-110.109551,-108.557890,-107.261807,-108.494681,1.000755,2
5,144.557798,1.085820,0.963000,0.135032,0.002058,{'selection__threshold': 0.0020583312},-107.725089,-107.782451,-109.816862,-107.867524,-106.535360,-107.945457,1.055627,1
6,146.369314,6.602300,1.113318,0.158732,0.003708,{'selection__threshold': 0.0037081626},-111.785424,-112.628683,-111.383284,-112.887282,-114.179844,-112.572903,0.971547,7
7,156.957478,0.869009,1.232444,0.143671,0.01621,{'selection__threshold': 0.01621042},-132.529109,-132.610008,-133.976569,-132.291876,-133.121972,-132.905907,0.599928,8


0.0020583312 is best

In [26]:
ssselected = model.feature_importances_ > (0.0020583312 - 10 ** -10)

In [27]:
t_data.loc[:,ssselected].columns

Index(['initial_time_seconds', 'max_overtime_minutes', 'game_duration_seconds',
       'bot_rating', 'a_points__sum_values', 'a_turn_type_Challenge__mean',
       'a_turn_type_Pass__sum_values', 'a_turn_type_Timeout__mean',
       'a_Z__mean', 'p_points__sum_values', 'p_points__median',
       'p_points__mean', 'p_points__standard_deviation',
       'p_points__root_mean_square', 'p_score__sum_values',
       'p_score__root_mean_square', 'p_score__maximum',
       'p_turn_type_End__mean', 'p_turn_type_Exchange__sum_values',
       'p_turn_type_Pass__mean', 'p_turn_type_Pass__standard_deviation',
       'p_turn_type_Play__mean', 'p_t__mean', 'p_A__maximum', 'p_I__maximum',
       'p_.__median', 'p_TileType.N__median',
       'p_TileType.W2__standard_deviation', 'p_move_len__mean',
       'p_move_len__root_mean_square', 'p_jokers_num__sum_values',
       'p_jokers_num__mean', 'p_special_tile__sum_values',
       'b_turn_type_End__mean', 'b_e__standard_deviation', 'b_l__mean',
       'b_U_

In [29]:
thresholds[2120-20:2120+30:10]

array([0.00167678, 0.00183573, 0.00205833, 0.00245118, 0.00370816],
      dtype=float32)

In [30]:
parameters3 = {'selection__threshold': thresholds[2120-20:2120+30:10]}


In [31]:
clf3 = GridSearchCV(s_reg_pipe, parameters3, n_jobs=2, scoring='neg_root_mean_squared_error')


In [32]:
clf3.fit(n_data, n_ratings)

GridSearchCV(estimator=Pipeline(steps=[('extract_first',
                                        ColumnsSetterTransformer(columns={'is_bot_first': <functions.is_bot_extractor.IsBotExtarctor object at 0x000001D7A68D2C70>})),
                                       ('hot',
                                        OneHotEncoderTransformer(columns={'bot_name': ['BetterBot',
                                                                                       'STEEBot',
                                                                                       'HastyBot'],
                                                                          'game_end_reason': array(['STANDARD', 'RESIGNED', 'TIME', 'CONSECUTIVE_ZEROES'], dtype=obj...
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     n_estimators=100,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     predictor=None,
                                                     random_state=0, ...))]),
             n_jobs=2,
             param_grid={'selection__threshold': array([0.00167678, 0.00183573, 0.00205833, 0.00245118, 0.00370816],
      dtype=float32)},
             scoring='neg_root_mean_squared_error')

In [33]:
pd.DataFrame(clf3.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_selection__threshold,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,150.361273,3.461405,1.001403,0.092913,0.001677,{'selection__threshold': 0.0016767832},-107.648911,-108.895245,-110.109551,-108.557890,-107.261807,-108.494681,1.000755,4
1,157.147170,6.349638,1.133310,0.091521,0.001836,{'selection__threshold': 0.001835726},-108.080335,-108.006952,-109.748618,-107.218407,-106.691949,-107.949252,1.036646,3
2,157.304995,2.051235,1.092951,0.068338,0.002058,{'selection__threshold': 0.0020583312},-107.725089,-107.782451,-109.816862,-107.867524,-106.535360,-107.945457,1.055627,2
3,158.053260,1.617717,1.116535,0.107854,0.002451,{'selection__threshold': 0.002451181},-106.778900,-108.240401,-107.630874,-107.066917,-106.262750,-107.195968,0.684321,1
4,146.517138,18.940866,1.054502,0.118648,0.003708,{'selection__threshold': 0.0037081626},-111.785424,-112.628683,-111.383284,-112.887282,-114.179844,-112.572903,0.971547,5


best is 0.002451181

In [36]:
thresholds[2130 - 20: 2130 + 15:2]

array([0.00183573, 0.00189468, 0.00192354, 0.00193979, 0.0020404 ,
       0.00205833, 0.00215331, 0.00225768, 0.00227384, 0.00235916,
       0.00245118, 0.00260127, 0.00279378, 0.00307217, 0.00324341,
       0.00370816, 0.00372989, 0.00464415], dtype=float32)

In [37]:
parameters4 = {'selection__threshold': thresholds[2130 - 20: 2130 + 15:2]}


In [38]:
clf4 = GridSearchCV(s_reg_pipe, parameters4, n_jobs=2, scoring='neg_root_mean_squared_error')


In [39]:
clf4.fit(n_data, n_ratings)

GridSearchCV(estimator=Pipeline(steps=[('extract_first',
                                        ColumnsSetterTransformer(columns={'is_bot_first': <functions.is_bot_extractor.IsBotExtarctor object at 0x000001D7A68D2C70>})),
                                       ('hot',
                                        OneHotEncoderTransformer(columns={'bot_name': ['BetterBot',
                                                                                       'STEEBot',
                                                                                       'HastyBot'],
                                                                          'game_end_reason': array(['STANDARD', 'RESIGNED', 'TIME', 'CONSECUTIVE_ZEROES'], dtype=obj...
                                                     num_parallel_tree=None,
                                                     predictor=None,
                                                     random_state=0, ...))]),
             n_jobs=2,
             param_grid={'selection__threshold': array([0.00183573, 0.00189468, 0.00192354, 0.00193979, 0.0020404 ,
       0.00205833, 0.00215331, 0.00225768, 0.00227384, 0.00235916,
       0.00245118, 0.00260127, 0.00279378, 0.00307217, 0.00324341,
       0.00370816, 0.00372989, 0.00464415], dtype=float32)},
             scoring='neg_root_mean_squared_error')

In [40]:
pd.DataFrame(clf4.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_selection__threshold,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,146.739235,0.485254,1.122801,0.102007,0.001836,{'selection__threshold': 0.001835726},-108.080335,-108.006952,-109.748618,-107.218407,-106.691949,-107.949252,1.036646,14
1,143.132236,2.623917,1.042202,0.055990,0.001895,{'selection__threshold': 0.0018946786},-107.428738,-108.774369,-108.618867,-107.391484,-106.254319,-107.693555,0.922728,5
2,140.882800,0.852587,1.003399,0.011359,0.001924,{'selection__threshold': 0.0019235439},-107.313801,-108.774369,-109.221485,-107.301206,-106.288262,-107.779825,1.071309,9
3,140.893400,0.780469,1.054805,0.069297,0.00194,{'selection__threshold': 0.0019397888},-107.313801,-108.774369,-108.876997,-107.385221,-106.288262,-107.727730,0.977444,7
4,142.000800,0.821961,0.990600,0.081431,0.00204,{'selection__threshold': 0.0020404023},-106.569889,-108.180207,-109.816862,-107.867524,-106.105416,-107.707980,1.308381,6
5,140.482799,0.666866,1.010601,0.084453,0.002058,{'selection__threshold': 0.0020583312},-107.725089,-107.782451,-109.816862,-107.867524,-106.535360,-107.945457,1.055627,13
6,140.330400,0.302402,1.012000,0.055569,0.002153,{'selection__threshold': 0.002153306},-107.632738,-108.077782,-108.928429,-107.120039,-106.237186,-107.599235,0.903514,3
7,140.936397,1.010663,1.010398,0.075177,0.002258,{'selection__threshold': 0.0022576763},-107.426653,-108.177404,-108.907132,-108.128950,-107.057054,-107.939438,0.643510,12
8,141.574800,1.673278,0.980400,0.074616,0.002274,{'selection__threshold': 0.0022738352},-107.426653,-108.618013,-108.907132,-108.128950,-106.256298,-107.867409,0.948976,10
9,139.062600,1.247091,1.048800,0.046568,0.002359,{'selection__threshold': 0.002359159},-106.731464,-108.646413,-108.982788,-108.128950,-105.575046,-107.612932,1.275996,4


0.002451181

In [42]:
thresholds[2130]

0.002451181

In [43]:
parameters5 = {'selection__threshold': thresholds[2130 - 5: 2130 + 5:1]}


In [44]:
clf5 = GridSearchCV(s_reg_pipe, parameters5, n_jobs=2, scoring='neg_root_mean_squared_error')


In [45]:
clf5.fit(n_data, n_ratings)

GridSearchCV(estimator=Pipeline(steps=[('extract_first',
                                        ColumnsSetterTransformer(columns={'is_bot_first': <functions.is_bot_extractor.IsBotExtarctor object at 0x000001D7A68D2C70>})),
                                       ('hot',
                                        OneHotEncoderTransformer(columns={'bot_name': ['BetterBot',
                                                                                       'STEEBot',
                                                                                       'HastyBot'],
                                                                          'game_end_reason': array(['STANDARD', 'RESIGNED', 'TIME', 'CONSECUTIVE_ZEROES'], dtype=obj...
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     n_estimators=100,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     predictor=None,
                                                     random_state=0, ...))]),
             n_jobs=2,
             param_grid={'selection__threshold': array([0.00225977, 0.00227384, 0.00229761, 0.00235916, 0.00241082,
       0.00245118, 0.00256867, 0.00260127, 0.00279277, 0.00279378],
      dtype=float32)},
             scoring='neg_root_mean_squared_error')

In [46]:
pd.DataFrame(clf5.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_selection__threshold,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,150.233376,2.312357,1.069000,0.034680,0.00226,{'selection__threshold': 0.00225977},-107.426653,-108.177404,-108.907132,-108.128950,-106.256298,-107.779287,0.894052,7
1,145.921997,0.278205,1.070401,0.030520,0.002274,{'selection__threshold': 0.0022738352},-107.426653,-108.618013,-108.907132,-108.128950,-106.256298,-107.867409,0.948976,8
2,147.065199,2.533206,1.033000,0.045677,0.002298,{'selection__threshold': 0.0022976073},-106.731464,-108.618013,-108.982788,-108.128950,-105.704585,-107.633160,1.230422,6
3,143.329000,1.220692,1.030599,0.017304,0.002359,{'selection__threshold': 0.002359159},-106.731464,-108.646413,-108.982788,-108.128950,-105.575046,-107.612932,1.275996,5
4,156.304931,9.118011,1.088599,0.065571,0.002411,{'selection__threshold': 0.002410817},-106.235722,-108.240401,-108.283255,-107.551051,-106.328505,-107.327787,0.892950,2
5,162.949107,1.101241,1.200634,0.047497,0.002451,{'selection__threshold': 0.002451181},-106.778900,-108.240401,-107.630874,-107.066917,-106.262750,-107.195968,0.684321,1
6,147.949512,6.082456,1.053801,0.065639,0.002569,{'selection__threshold': 0.0025686726},-106.471860,-108.330330,-107.630874,-107.116164,-107.323793,-107.374604,0.610476,3
7,160.858896,2.564101,1.176199,0.101576,0.002601,{'selection__threshold': 0.002601272},-106.471860,-108.330330,-109.166096,-107.116164,-106.675234,-107.551937,1.033100,4
8,162.253296,5.020921,1.024931,0.033763,0.002793,{'selection__threshold': 0.0027927663},-106.354460,-107.936141,-109.166096,-109.168742,-106.927686,-107.910625,1.144337,9
9,163.889758,2.107489,1.042402,0.143364,0.002794,{'selection__threshold': 0.002793783},-106.354460,-107.936141,-109.166096,-109.168742,-106.927686,-107.910625,1.144337,9


0.002451181

In [47]:
f_selected = model.feature_importances_ > (0.002451181 - 10 ** -10)

In [48]:
t_data.loc[:,f_selected].columns

Index(['initial_time_seconds', 'max_overtime_minutes', 'game_duration_seconds',
       'bot_rating', 'a_turn_type_Challenge__mean',
       'a_turn_type_Pass__sum_values', 'a_turn_type_Timeout__mean',
       'a_Z__mean', 'p_points__sum_values', 'p_points__median',
       'p_points__mean', 'p_points__root_mean_square',
       'p_score__root_mean_square', 'p_score__maximum',
       'p_turn_type_End__mean', 'p_turn_type_Exchange__sum_values',
       'p_turn_type_Pass__mean', 'p_turn_type_Pass__standard_deviation',
       'p_turn_type_Play__mean', 'p_A__maximum', 'p_I__maximum', 'p_.__median',
       'p_move_len__mean', 'p_move_len__root_mean_square',
       'p_jokers_num__sum_values', 'p_jokers_num__mean',
       'b_turn_type_End__mean', 'b_e__standard_deviation', 'b_l__mean',
       'b_move_len__sum_values', 'b_jokers_num__mean',
       'time_control_name_blitz', 'time_control_name_rapid',
       'time_control_name_regular', 'lexicon_CSW21', 'rating_mode_CASUAL',
       'bot_name_BetterBo

In [50]:
cv_res = cross_validate(XGBRegressor(random_state=0), t_data.loc[:,f_selected], n_ratings, cv=5,
                         scoring=('neg_root_mean_squared_error'),
                         return_train_score=True,)

In [51]:
cv_res

{'fit_time': array([2.51499629, 2.26700091, 2.16599703, 2.07200122, 2.37799954]),
 'score_time': array([0.01700044, 0.0119977 , 0.01099968, 0.0129993 , 0.01800036]),
 'test_score': array([-106.93082964, -107.45276004, -109.00666506, -107.82642914,
        -105.84051417]),
 'train_score': array([-85.83957996, -86.82917326, -86.76390578, -86.58867503,
        -86.69136323])}

In [52]:
cv_res = cross_validate(RandomForestRegressor(random_state=0), t_data.loc[:,f_selected], n_ratings, cv=5,
                         scoring=('neg_root_mean_squared_error'),
                         return_train_score=True,)

In [53]:
cv_res

{'fit_time': array([81.78741908, 80.12500334, 81.07307005, 79.90940499, 80.49954152]),
 'score_time': array([0.32302904, 0.31699705, 0.33902812, 0.32198834, 0.3119719 ]),
 'test_score': array([-103.98937585, -106.84101365, -106.68503619, -106.6518968 ,
        -105.0483003 ]),
 'train_score': array([-39.80210131, -39.39280216, -39.32628277, -39.39528381,
        -39.49469995])}

In [54]:
t_data.loc[:,f_selected]

,initial_time_seconds,max_overtime_minutes,game_duration_seconds,bot_rating,a_turn_type_Challenge__mean,a_turn_type_Pass__sum_values,a_turn_type_Timeout__mean,a_Z__mean,p_points__sum_values,p_points__median,...,b_l__mean,b_move_len__sum_values,b_jokers_num__mean,time_control_name_blitz,time_control_name_rapid,time_control_name_regular,lexicon_CSW21,rating_mode_CASUAL,bot_name_BetterBot,bot_name_HastyBot
1,1200,1,674.844274,1637,0.0,0.0,0.00000,0.037037,429.0,25.0,...,0.0,60.0,0.000000,0,0,1,0,1,1,0
3,900,5,492.268262,2071,0.0,0.0,0.00000,0.037037,440.0,28.0,...,0.0,57.0,0.076923,0,0,1,1,0,1,0
4,3600,1,350.861141,1936,0.0,0.0,0.00000,0.034483,119.0,5.5,...,0.0,76.0,0.066667,0,0,1,1,1,1,0
5,1200,1,642.688722,1844,0.0,0.0,0.00000,0.032258,325.0,20.5,...,0.0,67.0,0.133333,0,0,1,0,1,0,0
6,900,1,426.950541,2143,0.0,0.0,0.00000,0.040000,378.0,28.5,...,0.0,66.0,0.153846,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,3600,1,3669.988413,1952,0.0,0.0,0.03125,0.031250,340.0,16.5,...,0.0,73.0,0.062500,0,0,1,1,1,1,0
72770,3600,1,3662.659287,2237,0.0,0.0,0.03125,0.031250,97.0,6.0,...,0.0,79.0,0.062500,0,0,1,1,0,0,1
72771,900,10,236.133634,1614,0.0,0.0,0.00000,0.032258,390.0,20.5,...,0.0,59.0,0.133333,0,0,1,0,1,0,1
72772,900,1,681.370077,1674,0.0,0.0,0.00000,0.033333,388.0,19.0,...,0.0,64.0,0.000000,0,0,1,0,0,1,0


In [57]:
f_selected_50 = model.feature_importances_ > (thresholds[-50 - 1] - 10 ** -10)

In [58]:
t_data.loc[:,f_selected_50]

,initial_time_seconds,max_overtime_minutes,game_duration_seconds,bot_rating,a_points__sum_values,a_turn_type_Challenge__mean,a_turn_type_Pass__sum_values,a_turn_type_Timeout__mean,a_Z__mean,p_points__sum_values,...,b_jokers_num__sum_values,b_jokers_num__mean,time_control_name_blitz,time_control_name_rapid,time_control_name_regular,time_control_name_ultrablitz,lexicon_CSW21,rating_mode_CASUAL,bot_name_BetterBot,bot_name_HastyBot
1,1200,1,674.844274,1637,764.0,0.0,0.0,0.00000,0.037037,429.0,...,0.0,0.000000,0,0,1,0,0,1,1,0
3,900,5,492.268262,2071,758.0,0.0,0.0,0.00000,0.037037,440.0,...,1.0,0.076923,0,0,1,0,1,0,1,0
4,3600,1,350.861141,1936,597.0,0.0,0.0,0.00000,0.034483,119.0,...,1.0,0.066667,0,0,1,0,1,1,1,0
5,1200,1,642.688722,1844,752.0,0.0,0.0,0.00000,0.032258,325.0,...,2.0,0.133333,0,0,1,0,0,1,0,0
6,900,1,426.950541,2143,805.0,0.0,0.0,0.00000,0.040000,378.0,...,2.0,0.153846,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,3600,1,3669.988413,1952,754.0,0.0,0.0,0.03125,0.031250,340.0,...,1.0,0.062500,0,0,1,0,1,1,1,0
72770,3600,1,3662.659287,2237,687.0,0.0,0.0,0.03125,0.031250,97.0,...,1.0,0.062500,0,0,1,0,1,0,0,1
72771,900,10,236.133634,1614,783.0,0.0,0.0,0.00000,0.032258,390.0,...,2.0,0.133333,0,0,1,0,0,1,0,1
72772,900,1,681.370077,1674,830.0,0.0,0.0,0.00000,0.033333,388.0,...,0.0,0.000000,0,0,1,0,0,0,1,0


In [59]:
cv_res = cross_validate(RandomForestRegressor(random_state=0), t_data.loc[:,f_selected_50], n_ratings, cv=5,
                         scoring=('neg_root_mean_squared_error'),
                         return_train_score=True,)
cv_res

{'fit_time': array([115.88130188, 112.23748469, 109.5870297 , 109.74384522,
        114.1139338 ]),
 'score_time': array([0.33798122, 0.35002351, 0.30297279, 0.31200504, 0.33400917]),
 'test_score': array([-104.90205436, -106.98628126, -106.86542705, -106.25929734,
        -105.46530452]),
 'train_score': array([-39.8711378 , -39.46311746, -39.53815203, -39.52346491,
        -39.69217942])}

In [62]:
np.save('../feature_selection_consts/thresholds', thresholds)

In [3]:
np.load('../feature_selection_consts/thresholds.npy')

array([0.        , 0.        , 0.        , ..., 0.05297149, 0.06239389,
       0.06876279], dtype=float32)

In [66]:
selected_columns = t_data.loc[:,f_selected].columns

In [67]:
selected_columns

Index(['initial_time_seconds', 'max_overtime_minutes', 'game_duration_seconds',
       'bot_rating', 'a_turn_type_Challenge__mean',
       'a_turn_type_Pass__sum_values', 'a_turn_type_Timeout__mean',
       'a_Z__mean', 'p_points__sum_values', 'p_points__median',
       'p_points__mean', 'p_points__root_mean_square',
       'p_score__root_mean_square', 'p_score__maximum',
       'p_turn_type_End__mean', 'p_turn_type_Exchange__sum_values',
       'p_turn_type_Pass__mean', 'p_turn_type_Pass__standard_deviation',
       'p_turn_type_Play__mean', 'p_A__maximum', 'p_I__maximum', 'p_.__median',
       'p_move_len__mean', 'p_move_len__root_mean_square',
       'p_jokers_num__sum_values', 'p_jokers_num__mean',
       'b_turn_type_End__mean', 'b_e__standard_deviation', 'b_l__mean',
       'b_move_len__sum_values', 'b_jokers_num__mean',
       'time_control_name_blitz', 'time_control_name_rapid',
       'time_control_name_regular', 'lexicon_CSW21', 'rating_mode_CASUAL',
       'bot_name_BetterBo

In [68]:
np.save('../feature_selection_consts/columns', selected_columns)

In [4]:
np.load('../feature_selection_consts/columns.npy', allow_pickle=True)

array(['initial_time_seconds', 'max_overtime_minutes',
       'game_duration_seconds', 'bot_rating',
       'a_turn_type_Challenge__mean', 'a_turn_type_Pass__sum_values',
       'a_turn_type_Timeout__mean', 'a_Z__mean', 'p_points__sum_values',
       'p_points__median', 'p_points__mean', 'p_points__root_mean_square',
       'p_score__root_mean_square', 'p_score__maximum',
       'p_turn_type_End__mean', 'p_turn_type_Exchange__sum_values',
       'p_turn_type_Pass__mean', 'p_turn_type_Pass__standard_deviation',
       'p_turn_type_Play__mean', 'p_A__maximum', 'p_I__maximum',
       'p_.__median', 'p_move_len__mean', 'p_move_len__root_mean_square',
       'p_jokers_num__sum_values', 'p_jokers_num__mean',
       'b_turn_type_End__mean', 'b_e__standard_deviation', 'b_l__mean',
       'b_move_len__sum_values', 'b_jokers_num__mean',
       'time_control_name_blitz', 'time_control_name_rapid',
       'time_control_name_regular', 'lexicon_CSW21', 'rating_mode_CASUAL',
       'bot_name_BetterBo

In [71]:
t_data.loc[:,np.load('../feature_selection_consts/columns.npy', allow_pickle=True)]

,initial_time_seconds,max_overtime_minutes,game_duration_seconds,bot_rating,a_turn_type_Challenge__mean,a_turn_type_Pass__sum_values,a_turn_type_Timeout__mean,a_Z__mean,p_points__sum_values,p_points__median,...,b_l__mean,b_move_len__sum_values,b_jokers_num__mean,time_control_name_blitz,time_control_name_rapid,time_control_name_regular,lexicon_CSW21,rating_mode_CASUAL,bot_name_BetterBot,bot_name_HastyBot
1,1200,1,674.844274,1637,0.0,0.0,0.00000,0.037037,429.0,25.0,...,0.0,60.0,0.000000,0,0,1,0,1,1,0
3,900,5,492.268262,2071,0.0,0.0,0.00000,0.037037,440.0,28.0,...,0.0,57.0,0.076923,0,0,1,1,0,1,0
4,3600,1,350.861141,1936,0.0,0.0,0.00000,0.034483,119.0,5.5,...,0.0,76.0,0.066667,0,0,1,1,1,1,0
5,1200,1,642.688722,1844,0.0,0.0,0.00000,0.032258,325.0,20.5,...,0.0,67.0,0.133333,0,0,1,0,1,0,0
6,900,1,426.950541,2143,0.0,0.0,0.00000,0.040000,378.0,28.5,...,0.0,66.0,0.153846,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,3600,1,3669.988413,1952,0.0,0.0,0.03125,0.031250,340.0,16.5,...,0.0,73.0,0.062500,0,0,1,1,1,1,0
72770,3600,1,3662.659287,2237,0.0,0.0,0.03125,0.031250,97.0,6.0,...,0.0,79.0,0.062500,0,0,1,1,0,0,1
72771,900,10,236.133634,1614,0.0,0.0,0.00000,0.032258,390.0,20.5,...,0.0,59.0,0.133333,0,0,1,0,1,0,1
72772,900,1,681.370077,1674,0.0,0.0,0.00000,0.033333,388.0,19.0,...,0.0,64.0,0.000000,0,0,1,0,0,1,0
